# 🐝 HornetRL: Development Environment Setup

This notebook configures a persistent training environment on Google Colab for the HornetRL framework.

**Pipeline Overview:**
* **Persistence:** Mounts Google Drive to ensure model checkpoints and logs are saved permanently.
* **Version Control:** Clones the repository or pulls the latest updates if it already exists.
* **Environment:** Installs dependencies in `editable` mode to facilitate rapid iteration.
* **Execution:** Launches GPU-accelerated training and visualization routines.

In [ ]:
# 1. MOUNT GOOGLE DRIVE
from google.colab import drive
import os

print("--> Mounting Google Drive...")
drive.mount('/content/drive')

# 2. SETUP PROJECT DIRECTORY
# Ensure a dedicated directory exists for project organization
project_root = '/content/drive/MyDrive/HornetRL_Dev'

if not os.path.exists(project_root):
    print(f"--> Creating project folder at {project_root}...")
    os.makedirs(project_root)
else:
    print(f"--> Found existing project folder at {project_root}")

os.chdir(project_root)

In [ ]:
# 3. CLONE OR UPDATE REPOSITORY
repo_name = "hornetRL"

if not os.path.exists(repo_name):
    print("--> Cloning HornetRL repository...")
    !git clone https://github.com/lhooz/hornetRL.git
else:
    print("--> Repository already exists. Pulling latest changes...")
    os.chdir(repo_name)
    !git pull
    os.chdir('..') # Reset to project root

In [ ]:
# 4. INSTALL DEPENDENCIES
import os

# Path configuration
repo_path = os.path.join(project_root, repo_name)

# Verify directory structure
try:
    os.chdir(repo_path)
    print(f"--> Working Directory: {os.getcwd()}")
except FileNotFoundError:
    print("Error: Repo folder not found. Did the clone step finish?")

# Install package in editable mode (-e) to reflect changes immediately
print("--> Installing HornetRL package...")
!pip install -e .

In [ ]:
# 5. RUN TRAINING
# Configure output directory for checkpoints

ckpt_dir = os.path.join(repo_path, "checkpoints_shac")
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

print(f"--> Starting Training on GPU...")
print(f"--> Saving checkpoints to: {ckpt_dir}")

# Execute training module
# --gpu: Enables JAX GPU acceleration
# --dir: Specifies the output directory
!python -m hornetRL.train --gpu --dir "{ckpt_dir}"

In [ ]:
# 6. VISUALIZE POLICY
# Executes inference and renders the resulting trajectory GIF

import os
import glob
import re
import shutil
from IPython.display import Image, display

# Configuration check
if 'ckpt_dir' not in locals():
    ckpt_dir = "/content/drive/MyDrive/HornetRL_Dev/hornetRL/checkpoints_shac"

# 1. Checkpoint Resolution
checkpoints = glob.glob(os.path.join(ckpt_dir, "shac_params_*.pkl"))

if not checkpoints:
    print("⚠️ No checkpoints found! Ensure training (Step 5) has produced output.")
else:
    # Identify the latest checkpoint by step number (Safer Regex)
    latest_ckpt = max(checkpoints, key=lambda f: int(re.search(r"params_(\d+)", f).group(1)) if re.search(r"params_(\d+)", f) else 0)
    print(f"--> Loading latest checkpoint: {os.path.basename(latest_ckpt)}")

    # 2. Execute Inference (FORCE CPU)
    # We explicitly set JAX_PLATFORMS=cpu to avoid GPU overhead during sequential rendering
    !JAX_PLATFORMS=cpu python -m hornetRL.inference_hornet --checkpoint "{latest_ckpt}"

    # 3. Artifact Management
    # Archive the result with the step number and display it
    default_output = "hornet_flight_inference.gif"
    
    if os.path.exists(default_output):
        # Extract step number safely for renaming
        match = re.search(r"params_(\d+)", os.path.basename(latest_ckpt))
        step_num = match.group(1) if match else "final"
        
        new_name = os.path.join(ckpt_dir, f"replay_step_{step_num}.gif")
        
        shutil.copy(default_output, new_name)
        print(f"--> Flight Replay archived to: {new_name}")
        
        # Resize display to 600px width for better visibility
        display(Image(filename=default_output, width=600))
    else:
        print("--> Error: GIF generation failed. Check execution logs.")